Code to call the OpenAI API to call and solve visual puzzles (ConceptARC problems).

https://github.com/victorvikram/ConceptARC/tree/main/MinimalTasks


In [19]:
from openai import OpenAI
import dotenv
import os
from rich import print as rprint # for making fancy outputs

dotenv.load_dotenv()

client = OpenAI()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Call ConceptARC solver. Start by creating a good system prompt.

In [20]:
system_prompt = "You are an intelligent solver of puzzles."
user_prompt = 'your job is to solve a puzzle. These are puzzles in json format. I have some training examples and a test example in ConceptARC. Please solve it. Here is the puzzle:  {"train":[{"input":[[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,4,0,0,0,0,0],[0,0,0,4,4,4,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]],"output":[[0,4,0],[4,4,4]]},{"input":[[0,0,3,3,0,0,0,0],[0,0,3,3,0,0,0,0],[0,0,0,0,0,0,0,0],[0,4,0,4,0,0,0,0],[0,4,4,4,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]],"output":[[3,3],[3,3]]},{"input":[[0,0,0,0,0,0,0,0,0],[0,0,0,3,3,3,3,3,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,4,4,4,4,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],"output":[[3,3,3,3,3]]},{"input":[[0,0,0,0,0,0,0,0],[0,0,0,4,0,0,0,0],[0,0,4,0,4,0,0,0],[0,0,0,4,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,3,3,3,3,0,0],[0,0,3,0,0,3,0,0],[0,0,0,3,3,0,0,0]],"output":[[0,4,0],[4,0,4],[0,4,0]]}],"test":[{"input":[[4,4,4,0,0,0],[4,4,4,0,0,0],[0,0,0,0,0,0],[0,0,3,0,0,0],[3,3,3,3,0,0],[0,0,0,0,0,0]],"output":[[4,4,4],[4,4,4]]},{"input":[[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[4,4,4,4,4,4,4,4,4,4],[3,3,3,3,3,3,3,3,3,3],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]],"output":[[4,4,4,4,4,4,4,4,4,4]]},{"input":[[0,0,0,0,0],[0,3,3,3,0],[0,3,0,3,0],[0,3,3,3,0],[0,4,0,4,0],[4,0,4,0,4]],"output":[[3,3,3],[3,0,3],[3,3,3]]}]}'


Solve the problem now.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content": system_prompt},
        {"role": "user",  "content": user_prompt}
    ],
    temperature=1.0,
    max_tokens=1000,
)

print(response.choices[0].message.content) # print the response from the model


Use a vision model now to solve ConceptARC puzzle


In [ ]:
prompt = ("You are an intelligent solver of puzzles. " 
"Your job is to solve a puzzle. The puzzle is given to you as an image " 
"and you need to provide the solution in text format. "
"Please provide the solution in a clear and concise manner. "
"Make sure to include all necessary details in the solution. "
"Feel free to make any inferences you need to. "
)





Now call the OpenAI vision model API

In [ ]:
import base64
import requests
import json

function to encode the image

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string



In [ ]:
# Path to image file
image_path = "imgs/puzzle.png"

def get_image_caption(image_path, prompt):
  # Getting the base64 string
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 512
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  return response.json()['choices'][0]['message']['content']




Pass this to the solver

In [ ]:
caption = get_image_caption(image_path, prompt)

#print("Caption for the image:")
#print(caption)
rprint(caption) # print the caption

The puzzle presents a series of grids where colored shapes are arranged in various configurations. The goal is to 
deduce a pattern from these configurations and apply that to fill in the missing pieces.

### Solution Outline

1. **Identify Patterns in Grids:**
   - Each grid contains a combination of colored shapes placed in different positions.
   - The patterns seem to vary in colors and shapes across the grids.

2. **Analyze Shape Placement:**
   - Look for consistent positioning of shapes from one grid to the next.
   - Identify which colors or shapes appear in each grid and where they are located.

3. **Fill in Missing Shapes:**
   - For each grid, determine what shape/color would logically fit into the patterns observed in previous grids.
   - Follow the sequence seen to predict the next logical piece for the remaining shapes.

4. **Final Arrangement:**
   - The final arrangement should complete the pattern established in the previous grids.

### Result

The missing piece for each respective grid on the right should match the inferred patterns observed in the left 
grids. 

- **Grid 1:** Orange (square)
- **Grid 2:** Blue (small square)
- **Grid 3:** Orange (rectangle)
- **Grid 4:** Blue (small square)

In summary, the missing shapes/colors can be filled in according to the established patterns as described.